# HIERARCHICAL TRANSFORMERS PERSONALITY CLASSIFICATION

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 127.0 MB/s eta 0:00:00


In [ ]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from collections import defaultdict
from textwrap import wrap

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

### Setting up

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from google.colab import files
uploaded = files.upload()
df = pd.read_excel('essays.xlsx')
df.head()

Saving essays.xlsx to essays.xlsx


,#AUTHID,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,1997_504851.txt,"Well, right now I just woke up from a mid-day ...",n,y,y,n,y
1,1997_605191.txt,"Well, here we go with the stream of consciousn...",n,n,y,n,n
2,1997_687252.txt,An open keyboard and buttons to push. The thin...,n,y,n,y,y
3,1997_568848.txt,I can't believe it! It's really happening! M...,y,n,y,y,n
4,1997_688160.txt,"Well, here I go with the good old stream of co...",y,n,y,n,y


In [ ]:
personalities=["cEXT", "cNEU", "cAGR", "cCON", "cOPN"]

for personality in personalities:
    df[personality].replace(["y", "n"], [1, 0], inplace=True)

df.drop(columns='#AUTHID', inplace=True)
df.head()

,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,"Well, right now I just woke up from a mid-day ...",0,1,1,0,1
1,"Well, here we go with the stream of consciousn...",0,0,1,0,0
2,An open keyboard and buttons to push. The thin...,0,1,0,1,1
3,I can't believe it! It's really happening! M...,1,0,1,1,0
4,"Well, here I go with the good old stream of co...",1,0,1,0,1


In [ ]:
#Split each text longer than 200 words into multiple texts with an overlap of 50 words
import math

def split_text_into_segments(text, segment_length, overlap):
    words = text.split()
    segments = []
    for i in range(0, len(words), segment_length - overlap):
        segment = " ".join(words[i:i+segment_length])
        segments.append(segment)
    return segments

def split_long_texts(df, max_length, overlap):
    new_rows = []
    for index, row in df.iterrows():
        text = row['TEXT']
        atr_cols = row[['cEXT', 'cNEU', 'cAGR', 'cCON', 'cOPN']]
        segments = split_text_into_segments(text, max_length, overlap)
        for segment in segments:
            new_row = {'text': segment}
            new_row.update(atr_cols)
            new_rows.append(new_row)
    new_df = pd.DataFrame(new_rows)
    return new_df

max_length = 200  # Maximum length of each segment in words
overlap = 50  # Number of overlapping words between segments

split_df = split_long_texts(df, max_length, overlap)
split_df.head()

,text,cEXT,cNEU,cAGR,cCON,cOPN
0,"Well, right now I just woke up from a mid-day ...",0,1,1,0,1
1,"to correct the past, but I don't really know h...",0,1,1,0,1
2,I don't have a worry in the world. The only th...,0,1,1,0,1
3,"keep my priorities straight. Living at home, I...",0,1,1,0,1
4,put more pressure on me to do better in school...,0,1,1,0,1


In [ ]:
split_df.info()
split_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11960 entries, 0 to 11959
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    11960 non-null  object
 1   cEXT    11960 non-null  int64 
 2   cNEU    11960 non-null  int64 
 3   cAGR    11960 non-null  int64 
 4   cCON    11960 non-null  int64 
 5   cOPN    11960 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 560.8+ KB


,cEXT,cNEU,cAGR,cCON,cOPN
count,11960.000000,11960.000000,11960.000000,11960.000000,11960.000000
mean,0.517559,0.508278,0.529599,0.511957,0.521572
std,0.499712,0.499952,0.499144,0.499878,0.499555
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
RANDOM_SEED = 72
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
#An example of what will be performed on big scale
sample_txt='Personal Health Record (Extract)\nCreated on October 24, 2019\nPatient\nSteven Fuerst\nBirthdate\nDecember 10, 1979\nRace\nInformation not\navailable'

tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
encoding = tokenizer.encode_plus(
                  sample_txt,
                  max_length=128,
                  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
                  return_token_type_ids=False,
                  pad_to_max_length=True,
                  return_attention_mask=True,
                  return_tensors='pt',  # Return PyTorch tensors
                )
print(encoding)              

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'input_ids': tensor([[  101, 13907,  3225,  7992,   113, 18684,  5822,   114, 25423,  1113,
          1357,  1572,   117, 10351,  7195,  9080,  6536, 14763,  1468,  1204,
         20729,  9216,  1382,  1275,   117,  2333,  6398,  4219,  1136,  1907,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


#### Datasets

In [ ]:
class EssayDataset(Dataset):
    
    def __init__(self, doc, targets, tokenizer, max_len):
        self.doc = doc
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.doc)
  
    def __getitem__(self, item):
        doc = str(self.doc[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
                    doc,
                    add_special_tokens=True,
                    max_length=self.max_len,
                    return_token_type_ids=False,
                    pad_to_max_length=True,
                    return_attention_mask=True,
                    return_tensors='pt',
                    )

        return {
            'doc_text': doc,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
            }


def create_data_loader(df, tokenizer, max_len, batch_size, personality):
    ds = EssayDataset(
        doc=df.text.to_numpy(),
        targets=df[personality].to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
      )
    
    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=2
        )
    

In [ ]:
df_train, df_test = train_test_split(split_df, test_size=0.1, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)

In [ ]:
MAX_LEN = 256                                                       #MORA LI BITI POTENCIJA OD 2
BATCH_SIZE = 16

#personalities=["cEXT", "cNEU", "cAGR", "cCON", "cOPN"]

#cEXT
train_data_loader_ext = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE, "cEXT")
val_data_loader_ext = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE, "cEXT")
test_data_loader_ext = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE, "cEXT")

#cNEU
train_data_loader_neu = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE, "cNEU")
val_data_loader_neu = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE, "cNEU")
test_data_loader_neu = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE, "cNEU")

#cAGR
train_data_loader_agr = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE, "cAGR")
val_data_loader_agr = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE, "cAGR")
test_data_loader_agr = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE, "cAGR")

#cCON
train_data_loader_con = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE, "cCON")
val_data_loader_con = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE, "cCON")
test_data_loader_con = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE, "cCON")

#cOPN
train_data_loader_opn = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE, "cOPN")
val_data_loader_opn = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE, "cOPN")
test_data_loader_opn = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE, "cOPN")

### Model Definition

In [ ]:
class PersonalityClassifier(nn.Module):
    def __init__(self, n_classes):
        super(PersonalityClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask,
          return_dict=False
        )
        
        output = self.drop(pooled_output)
        return self.out(output)

##### Extraversion

In [ ]:
data_ext = next(iter(train_data_loader_ext))

model_ext = PersonalityClassifier(2)  #2 is number of classes (binary classifier for a personality)
model_ext = model_ext.to(device)

input_ids = data_ext['input_ids'].to(device)
attention_mask = data_ext['attention_mask'].to(device)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight

##### Neuroticism

In [ ]:
data_neu = next(iter(train_data_loader_neu))

model_neu = PersonalityClassifier(2)  #2 is number of classes (binary classifier for a personality)
model_neu = model_neu.to(device)

input_ids = data_neu['input_ids'].to(device)
attention_mask = data_neu['attention_mask'].to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


##### Agreeableness

In [ ]:
data_agr = next(iter(train_data_loader_agr))

model_agr = PersonalityClassifier(2)  #2 is number of classes (binary classifier for a personality)
model_agr = model_agr.to(device)

input_ids = data_agr['input_ids'].to(device)
attention_mask = data_agr['attention_mask'].to(device)

##### Consciousness

In [ ]:
data_con = next(iter(train_data_loader_con))

model_con = PersonalityClassifier(2)  #2 is number of classes (binary classifier for a personality)
model_con = model_con.to(device)

input_ids = data_con['input_ids'].to(device)
attention_mask = data_con['attention_mask'].to(device)

##### Openness

In [ ]:
data_opn = next(iter(train_data_loader_opn))

model_opn = PersonalityClassifier(2)  #2 is number of classes (binary classifier for a personality)
model_opn = model_opn.to(device)

input_ids = data_opn['input_ids'].to(device)
attention_mask = data_opn['attention_mask'].to(device)

### Training

In [ ]:
def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device, 
  scheduler, 
  n_examples
    ):
    model = model.train()

    losses = []
    correct_predictions = 0

    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        outputs = model(
          input_ids=input_ids,
          attention_mask=attention_mask
        )

        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)

        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()

    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
            )
            _, preds = torch.max(outputs, dim=1)

            loss = loss_fn(outputs, targets)

            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)

def get_predictions(model, data_loader):
    model = model.eval()

    review_texts = []
    predictions = []
    prediction_probs = []
    real_values = []

    with torch.no_grad():
        for d in data_loader:

            texts = d["doc_text"]
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
            )
            _, preds = torch.max(outputs, dim=1)

            probs = F.softmax(outputs, dim=1)

            review_texts.extend(texts)
            predictions.extend(preds)
            prediction_probs.extend(probs)
            real_values.extend(targets)

    predictions = torch.stack(predictions).cpu()
    prediction_probs = torch.stack(prediction_probs).cpu()
    real_values = torch.stack(real_values).cpu()
    return review_texts, predictions, prediction_probs, real_values

def save_model(model, name):
  path_pt = F"/content/gdrive/My Drive/Colab Notebooks/models/{name}.pt"
  path_bin = F"/content/gdrive/My Drive/Colab Notebooks/models/{name}.bin" 
  torch.save(model.state_dict(), path_pt)
  torch.save(model.state_dict(), path_bin)

In [ ]:
EPOCHS = 10

In [ ]:
def my_transformer_training(model, train_data_loader, val_data_loader, test_data_loader, personality):  
  optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
  total_steps = len(train_data_loader) * EPOCHS

  scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
  )

  loss_fn = nn.CrossEntropyLoss().to(device)


  history = defaultdict(list)
  best_accuracy = 0

  for epoch in range(EPOCHS):

      print(f'Epoch {epoch + 1}/{EPOCHS}')
      print('-' * 10)

      train_acc, train_loss = train_epoch(
      model,
      train_data_loader,    
      loss_fn, 
      optimizer, 
      device, 
      scheduler, 
      len(df_train)
      )

      print(f'Train loss {train_loss} accuracy {train_acc}')

      val_acc, val_loss = eval_model(
      model,
      val_data_loader,
      loss_fn, 
      device, 
      len(df_val)
      )

      print(f'Val   loss {val_loss} accuracy {val_acc}')
      print()

      history['train_acc'].append(train_acc)
      history['train_loss'].append(train_loss)
      history['val_acc'].append(val_acc)
      history['val_loss'].append(val_loss)

      if val_acc > best_accuracy:
          save_model(model, 'model_'+personality)
          best_accuracy = val_acc

  test_acc, _ = eval_model(
        model,
        test_data_loader,
        loss_fn,
        device,
        len(df_test)
      )
  print('\nTest Accuracy:\n')
  print(test_acc.item())

  y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  test_data_loader
  )

  f1 = f1_score(y_test, y_pred)
  print("F1 score:", f1)
  print(classification_report(y_test, y_pred, target_names=['no', 'yes']))

In [ ]:
import warnings
warnings.filterwarnings('ignore')

##### Extraversion

In [ ]:
my_transformer_training(model_ext, train_data_loader_ext, val_data_loader_ext, test_data_loader_ext, 'ext')

Epoch 1/10
----------


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Train loss 0.7092646113299331 accuracy 0.5157933853586028


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Val   loss 0.7062871409089941 accuracy 0.5016722408026756

Epoch 2/10
----------


...

Val   loss 2.217042054000654 accuracy 0.7023411371237458



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Test Accuracy:

0.705685618729097


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


F1 score: 0.7105263157894737
              precision    recall  f1-score   support

          no       0.73      0.67      0.70       306
         yes       0.68      0.74      0.71       292

    accuracy                           0.71       598
   macro avg       0.71      0.71      0.71       598
weighted avg       0.71      0.71      0.71       598



##### Neuroticism

In [ ]:
my_transformer_training(model_neu, train_data_loader_neu, val_data_loader_neu, test_data_loader_neu, 'neu')

Epoch 1/10
----------
Train loss 0.7110793531206706 accuracy 0.510776662950576
Val   loss 0.7029258404907427 accuracy 0.4899665551839465

Epoch 2/10
----------
Train loss 0.701769827202206 accuracy 0.5030657748049052
Val   loss 0.6933733303295938 accuracy 0.4899665551839465

Epoch 3/10
----------
Train loss 0.7221964618072113 accuracy 0.4982348569305091
Val   loss 0.6950547742216211 accuracy 0.4899665551839465

Epoch 4/10
----------
Train loss 0.7146121616880748 accuracy 0.5078966926793014
Val   loss 0.6960941098238292 accuracy 0.4899665551839465

Epoch 5/10
----------
Train loss 0.7094054312443839 accuracy 0.5052954292084727
Val   loss 0.6974547693603917 accuracy 0.4899665551839465

Epoch 6/10
----------
Train loss 0.7000314820043045 accuracy 0.5207172054998143
Val   loss 0.6836582437941903 accuracy 0.5484949832775919

Epoch 7/10
----------
Train loss 0.6756480939320966 accuracy 0.5906726124117428
Val   loss 0.7005111904520738 accuracy 0.5501672240802675

Epoch 8/10
----------
Train l

##### Agreeableness

In [ ]:
my_transformer_training(model_agr, train_data_loader_agr, val_data_loader_agr, test_data_loader_agr, 'agr')

##### Consciousness

In [ ]:
my_transformer_training(model_con, train_data_loader_con, val_data_loader_con, test_data_loader_con, 'con')

##### Openness

In [ ]:
my_transformer_training(model_opn, train_data_loader_opn, val_data_loader_opn, test_data_loader_opn, 'opn')